**1. IMPORT LIBRARIES AND CIFAR 10, PREPARE DATA**

In [0]:
# general
import numpy as np
import cv2
import matplotlib.pyplot as plt
% matplotlib inline


# dataset
from keras.datasets import cifar10

# keras
from keras import Sequential
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from keras.optimizers import Adam, SGD
from keras.utils.np_utils import to_categorical
from keras.applications import vgg16
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input  # different expert has different method to preprocess data, importing this is very important

Using TensorFlow backend.


In [0]:
# split data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 110s 1us/step


In [0]:
# get labels
labels = []
for y in y_train.flatten():
  if y not in labels[:3000]:
    labels.append(y)
    
num_class = len(labels)
    
print(labels)
print(len(labels))

[6, 9, 4, 1, 2, 7, 8, 3, 5, 0]
10


In [0]:
# convert to one-hot encoded labels

y_train = to_categorical(y_train, num_class)
y_test = to_categorical(y_test, num_class)

In [0]:
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

In [0]:
# Check data dimensions
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3), (50000, 10), (10000, 10))

**3. BUILD FINE-TUNED VGG16 MODEL**

####  * Check Original VGG16 Model Structure



In [0]:
# original vgg model

origin_vgg = vgg16.VGG16()
origin_vgg.summary()

Instructions for updating:
Colocations handled automatically by placer.
553467904/553467096 [==============================] - 234s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_

####  * Drop Off The Top 4 Layers of VGG16 Model



In [0]:
vgg_model = vgg16.VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=(32,32,3))

58892288/58889256 [==============================] - 5s 0us/step


In [0]:
vgg_model.summary()
len(vgg_model.layers)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

19

####  * Rebuild the Top Layer And Combine With The Previous Layers

In [0]:
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(10, activation='softmax'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
top_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
Total params: 133,898
Trainable params: 133,898
Non-trainable params: 0
_________________________________________________________________


In [0]:
model = Model(inputs= vgg_model.input, outputs= top_model(vgg_model.output))

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

#### * Freeze All Layers Except The Top Layer

In [0]:
for layer in model.layers[:-1]:
    layer.trainable = False

**3. Train The Top Layer**

In [0]:
epochs = 50
batch_size = 32
lr = 0.0001

In [0]:
model.compile(SGD(lr = lr, momentum=0.9, nesterov=True), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [0]:
history = model.fit(x_train, 
                    y_train, 
                    epochs = epochs, 
                    batch_size = batch_size, 
                    verbose = 2,
                    validation_data = (x_test, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
 - 36s - loss: 5.5023 - acc: 0.4079 - val_loss: 1.7491 - val_acc: 0.5315
Epoch 2/50
 - 33s - loss: 1.8661 - acc: 0.4698 - val_loss: 1.3541 - val_acc: 0.5503
Epoch 3/50
 - 33s - loss: 1.5221 - acc: 0.5034 - val_loss: 1.2699 - val_acc: 0.5660
Epoch 4/50
 - 33s - loss: 1.4087 - acc: 0.5199 - val_loss: 1.2204 - val_acc: 0.5793
Epoch 5/50
 - 33s - loss: 1.3418 - acc: 0.5397 - val_loss: 1.1900 - val_acc: 0.5845
Epoch 6/50
 - 33s - loss: 1.3038 - acc: 0.5505 - val_loss: 1.1623 - val_acc: 0.5972
Epoch 7/50
 - 33s - loss: 1.2605 - acc: 0.5624 - val_loss: 1.1464 - val_acc: 0.6025
Epoch 8/50
 - 33s - loss: 1.2344 - acc: 0.5704 - val_loss: 1.1316 - val_acc: 0.6039
Epoch 9/50
 - 33s - loss: 1.2106 - acc: 0.5779 - val_loss: 1.1197 - val_acc: 0.6132
Epoch 10/50
 - 33s - loss: 1.1923 - acc: 0.5818 - val_loss: 1.1102 - val_acc: 0.6149
Epoch 11/50
 - 33s - loss: 1.1719 - acc: 0.5860 - val_loss: 1